# sw_audio_dsp pipeline designer!

You have an xcore and now you want DSP? In that case you are in the right place, this jupyter notebook will fulfill all of your DSP needs!

Below you will find 3 cells which must be modified and executed in order to make the DSP pipeline of your dreams a reality.

1. Immediately below this cell is the pipeline design cell. Here you must break the DSP pipeline down into threads and use the provided DSP stages to create a pipeline. Running this cell will product a diagram showing your pipeline. Make sure to capture each stage in your pipeline as a variable as it will be needed in the next step.
2. The cell after this is the tuning cell. First time through this can be ignored, but once your pipeline is running on a connected device then this cell can be updated and executed to updated each pipeline stage live.
3. The build and run stage. This final stage generates an application which uses your pipeline and uses xrun to get it running on a connected device. The application which is created will have the tuning parameters baked in.

Note that every time the pipeline cell is changed, the app must be regenerated before the tuning stage will work correctly as the stage indices used for communication will have changed.

In [ ]:
# Pipeline design stage

from audio_dsp.design.pipeline import Pipeline
from audio_dsp.stages.biquad import Biquad
from audio_dsp.stages.cascaded_biquads import CascadedBiquads

# 4 inputs
p = Pipeline(4)

with p.add_thread() as t:
    butter = t.stage(CascadedBiquads, p.i[:2])
with p.add_thread() as t:
    notch = t.stage(Biquad, butter.o)


p.set_outputs(notch.o)
p.draw()


In [ ]:
# Tuning stage

butter.make_butterworth_highpass(14, 2000)
notch.make_notch(4000, 5)


from audio_dsp.design.pipeline import send_config_to_device
send_config_to_device(p, "host_cmd_map/build/xvf_host")

In [ ]:
# Build and run

from audio_dsp.design.pipeline import generate_dsp_main
from build_utils import build_and_try_run
generate_dsp_main(p)
build_and_try_run()